<a href="https://colab.research.google.com/github/newlei/shortcut_code/blob/main/log_uniform_candidate_sampler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import tensorflow as tf
tf.__version__
import numpy as np

def gen_neg(num_sampled):
    SEED = 42 
    context_class = tf.reshape(tf.constant(0, dtype="int64"), (1, 1))#正样本id为0
    negative_sampling_candidates, true_expected_count, sampled_expected_count = tf.random.log_uniform_candidate_sampler(
        true_classes=context_class, # class that should be sampled as 'positive'
        num_true=1, # each positive skip-gram has 1 positive context class
        num_sampled=num_sampled, # number of negative context words to sample
        unique=True, # all the negative samples should be unique
        range_max=8, # pick index of the samples from [0, vocab_size]
        seed=SEED, # seed for reproducibility
        name="negative_sampling" # name of this operation
    ) 
    print(true_expected_count.numpy(),sampled_expected_count.numpy(),negative_sampling_candidates.numpy())

#计算P(class)，方便对应。
prob_list=[]
for i in range(8): 
    pro_one = (np.log(i + 2) - np.log(i + 1)) / np.log(8 + 1) 
    prob_list.append(pro_one)
print('P(class)',prob_list)

#采样1个负样本的时候。
print('------------num_sampled==1---------------')
for i in range(10):
    #可以看到true_expected_count都是P(0)=0.31546488的值。
    #sampled_expected_count是采样得到的负样本，概率就等于对应的P(class)，就是P(negative_sampling_candidates)。
    #采样得到的负样本negative_sampling_candidates，可以是正样本这里没有做区分，如果需要不包含正样本tf是在外层处理的。如：tf.nn.sampled_softmax_loss在采样结束后有对应的处理代码。
    gen_neg(1)

print('------------num_sampled==2---------------')
#采样2个负样本的时候。
for i in range(10):
    #可以看到true_expected_count是P(0)*num_sampled=0.31546488*2=0.63092976的附近。所以存在一个波动。即：\hat{P}(0)=[P(0)+波动]+[P(0)+波动]
    #sampled_expected_count是采样得到的负样本，概率是P(class)*num_sampled的附近，也是有个波动的。
    gen_neg(2)

print('------------num_sampled==3---------------')
#采样2个负样本的时候。
for i in range(10):
    #可以看到true_expected_count是P(0)*num_sampled=0.31546488*3=0.94639464的附近。即：\hat{P}(0)=[P(0)+波动]+[P(0)+波动]+[P(0)+波动]。因为num_sampled变多了，波动影响也就更大。\hat{P}(0)的浮动就大了
    #sampled_expected_count是采样得到的负样本，概率是P(class)*num_sampled的附近，波动也就更大
    gen_neg(3)

P(class) [0.3154648767857287, 0.18453512321427132, 0.13092975357145736, 0.10155700678750612, 0.08297811642676516, 0.0701569977949824, 0.06077275577647497, 0.053605369642814]
------------num_sampled==1---------------
[[0.31546488]] [0.08297811] [4]
[[0.31546488]] [0.18453512] [1]
[[0.31546488]] [0.18453512] [1]
[[0.31546488]] [0.18453512] [1]
[[0.31546488]] [0.08297811] [4]
[[0.31546488]] [0.31546488] [0]
[[0.31546488]] [0.13092975] [2]
[[0.31546488]] [0.18453512] [1]
[[0.31546488]] [0.13092975] [2]
[[0.31546488]] [0.13092975] [2]
------------num_sampled==2---------------
[[0.63092977]] [0.36907023 0.2618595 ] [1 2]
[[0.63092977]] [0.16595623 0.36907023] [4 1]
[[0.63092977]] [0.2618595  0.63092977] [2 0]
[[0.63092977]] [0.140314   0.20311402] [5 3]
[[0.63092977]] [0.63092977 0.36907023] [0 1]
[[0.63092977]] [0.16595623 0.140314  ] [4 5]
[[0.63092977]] [0.36907023 0.2618595 ] [1 2]
[[0.63092977]] [0.16595623 0.20311402] [4 3]
[[0.63092977]] [0.2618595  0.63092977] [2 0]
[[0.67923486]] [0